# Secural Resonances Identification

The notebook demonstrates how to identify whether the resonant argument (secular resonance) librates.

In [89]:
%%capture
%pip install pandas requests

In [88]:
import base64
import requests
import os
import pandas as pd


In [61]:
db_path = 'input/secular/res_status'
image_pattern = "input/secular/images/res_osc_{}.png"

statuses = {
    '0': 'non-resonant',
    '1': 'transient',
    '2': 'pure',
}

In [57]:
def format_number(num):
    return str(num).zfill(7)

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


In [60]:
df = pd.read_csv(db_path, sep='\s+', usecols=[0, 1, 2, 3, 4, 5, 6], 
                 names=['number', 'a', 'e', 'i', 'H', 's', 'label'], 
                 dtype={'number': str, 'a': float, 'e': float, 'i': float, 'H': float, 's': float, 'label': int})
df.head(5)

,number,a,e,i,H,s,label
0,183,2.792859,0.246347,0.470175,9.5,-50.402130,1
1,246,2.694623,0.107090,0.265736,8.5,-50.158177,2
2,724,2.455457,0.240730,0.199480,13.9,-50.274701,0
3,754,2.987385,0.060366,0.410754,9.3,-50.354245,2
4,1098,2.688798,0.094959,0.245192,10.5,-50.245275,2


In [74]:
images = {}
for number in df.head(1000)['number'].to_list():
    image = "input/secular/images/res_osc_{}.png".format(format_number(number))
    images[str(number)] = image
print(f"Total images: {len(images)}")
images

Total images: 100


{'183': 'input/secular/images/res_osc_0000183.png',
 '246': 'input/secular/images/res_osc_0000246.png',
 '724': 'input/secular/images/res_osc_0000724.png',
 '754': 'input/secular/images/res_osc_0000754.png',
 '1098': 'input/secular/images/res_osc_0001098.png',
 '1106': 'input/secular/images/res_osc_0001106.png',
 '1230': 'input/secular/images/res_osc_0001230.png',
 '1425': 'input/secular/images/res_osc_0001425.png',
 '1473': 'input/secular/images/res_osc_0001473.png',
 '1833': 'input/secular/images/res_osc_0001833.png',
 '1886': 'input/secular/images/res_osc_0001886.png',
 '1945': 'input/secular/images/res_osc_0001945.png',
 '2257': 'input/secular/images/res_osc_0002257.png',
 '2337': 'input/secular/images/res_osc_0002337.png',
 '2351': 'input/secular/images/res_osc_0002351.png',
 '2388': 'input/secular/images/res_osc_0002388.png',
 '2477': 'input/secular/images/res_osc_0002477.png',
 '2649': 'input/secular/images/res_osc_0002649.png',
 '2786': 'input/secular/images/res_osc_0002786.png

The prompt used in the LLM

In [75]:
prompt = '''
I want you to act a scientist–astronomer. You will get an image uploaded. The image contains the plot of the resonant angle of an asteroid vs time (from 0 to 2; the dimension is 10 Myr). The limits of OY axis are 0 and 360 degrees. The resonant angle cannot exceed these limits.

It is known that if the resonant angle librates, then the asteroid is trapped in the resonance. Librations mean oscillations, like sine. It means that the curve is within some limits (i.e., from 20 to 200) and does not cross the borders (0 and 360). It may come really close. 

The opposite situation is when the resonant angle circulates. It means that the curve is not limited and can reach the borders of the plot. In our case, if the resonant angle is greater than 360 or less than 0, then we add or substract 360 to the resonant angle to make it within the limits. Therefore, in the case of circulation, the pattern will be like linear curves parallel each other and following each other, crossing, for example, the top limit and then appearing in the bottom.

I want you to assess visually whether the resonant angle librates if you were a human looking at this image.

There are three possible cases:

1. The resonant angle librates all the time (from 0 to 2). Then you should reply 'pure'.
2. The resonant angle could librate some significant time, but in other time is circulates. Let's assume that by significant I mean 1.0 (10 Myr in the given scale). In this case, you should write 'transient'.
3. Otherwise, when the resonant angle circulates most of the time, please write 'non-resonant'.

As output, I want you only to print one word: pure, transient, or non-resonant. If you are not sure, write 'I do not know'. You will get tips if you perform the identification correctly.
'''

The code that runs the prompt above on the images generated earlier. Please add your OpenAI API key below to run the notebook.

In [77]:
api_key = os.environ.get('OPENAI_API_KEY', "DEFAULT-KEY-IF-NOT-ENV")

results = []
for num, image in images.items():
  base64_image = encode_image(image)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 500
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  data = response.json()
  # print(response.json())
  try:
    print(f"{num},{data.get('choices')[0].get('message').get('content')},{statuses[str(df.loc[df['number'] == num, 'label'].values[0])]}")
    results.append({'num': num, 'llm': data.get('choices')[0].get('message').get('content'), 'human': statuses[str(df.loc[df['number'] == num, 'label'].values[0])]})
  except:
    print(f"Error with the asteroid {num}. Try again with it later!")


183,transient,transient
246,pure,pure
724,non-resonant,non-resonant
754,pure,pure
1098,pure,pure
1106,transient,transient
1230,transient,non-resonant
1425,pure,pure
1473,pure,pure
1833,pure,pure
1886,non-resonant,non-resonant
1945,transient,non-resonant
2257,transient,transient
2337,non-resonant,non-resonant
2351,non-resonant,non-resonant
2388,transient,non-resonant
2477,transient,transient
2649,transient,non-resonant
2786,non-resonant,non-resonant
3104,transient,transient
Error with the asteroid 3528. Try again with it later!
3617,transient,transient
3815,pure,pure
3995,pure,pure
4046,non-resonant,non-resonant
4313,non-resonant,non-resonant
4452,transient,transient
4567,transient,transient
4572,pure,pure
4591,non-resonant,non-resonant
4766,transient,non-resonant
4974,pure,transient
5104,pure,pure
5105,transient,non-resonant
5122,pure,pure
5152,transient,non-resonant
5255,pure,pure
5270,pure,pure
5288,pure,pure
5489,non-resonant,non-resonant
5521,pure,transient
5611,transient,non-reson

In [80]:
df_res = pd.DataFrame(results)
df_res.head(5)

,num,llm,human
0,183,transient,transient
1,246,pure,pure
2,724,non-resonant,non-resonant
3,754,pure,pure
4,1098,pure,pure


In [82]:
equal_count = len(df_res[df_res['llm'] == df_res['human']])
not_equal_count = len(df_res[df_res['llm'] != df_res['human']])
total_count = len(df_res)
equal_ratio = (equal_count / total_count) * 100
not_equal_ratio = (not_equal_count / total_count) * 100

print("Number of rows where llm is equal to 'human':", equal_count)
print("Number of rows where llm is not equal to 'human':", not_equal_count)
print("Ratio of rows where llm is equal to 'human' to the total number: {:.2f}%".format(equal_ratio))
print("Ratio of rows where llm is not equal to 'human' to the total number: {:.2f}%".format(not_equal_ratio))

Number of rows where llm is equal to 'human': 74
Number of rows where llm is not equal to 'human': 25
Ratio of rows where llm is equal to 'human' to the total number: 74.75%
Ratio of rows where llm is not equal to 'human' to the total number: 25.25%


In [83]:
df_res[df_res['llm'] != df_res['human']].head(100)

,num,llm,human
6,1230,transient,non-resonant
11,1945,transient,non-resonant
15,2388,transient,non-resonant
17,2649,transient,non-resonant
29,4766,transient,non-resonant
30,4974,pure,transient
32,5105,transient,non-resonant
34,5152,transient,non-resonant
39,5521,pure,transient
40,5611,transient,non-resonant


In [85]:
df_tp = df_res[(df_res['llm'] == 'pure') & (df_res['human'] == 'pure')]
df_tn = df_res[(df_res['llm'] != 'pure') & (df_res['human'] != 'pure')]
df_fp = df_res[(df_res['llm'] == 'pure') & (df_res['human'] != 'pure')]
df_fn = df_res[(df_res['llm'] != 'pure') & (df_res['human'] == 'pure')]

total_count = len(df_res)

accuracy = (len(df_tp) + len(df_tn)) / total_count
precision = len(df_tp) / (len(df_tp) + len(df_fp))
recall = len(df_tp) / (len(df_tp) + len(df_fn))
f1_score = 2 * (precision * recall) / (precision + recall)

print("TP: {}, TN: {}, FP: {}, FN: {}".format(len(df_tp), len(df_tn), len(df_fp), len(df_fn)))
print("Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}".format(accuracy, precision, recall, f1_score))


TP: 42, TN: 51, FP: 5, FN: 1
Accuracy: 0.94, Precision: 0.89, Recall: 0.98, F1 Score: 0.93


In [86]:
df_fp.head(100)

,num,llm,human
30,4974,pure,transient
39,5521,pure,transient
65,8493,pure,non-resonant
80,10330,pure,non-resonant
84,11077,pure,non-resonant


In [87]:
df_fn.head(100)

,num,llm,human
61,8118,transient,pure
